<a href="https://colab.research.google.com/github/femketenharkel/Predicting_Ratings/blob/main/Baseline2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google colab version
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
# Load df Google Colab (feature engineered, encoded, scaled/unscaled)
import pandas as pd
df_final = pd.read_csv("/content/drive/My Drive/Thesis/Data/sample_df.csv")


In [27]:
pd.set_option('display.max_columns', None)
df_final.head()

,UserID,MovieID,Rating,Year,Month,Day,Hour,Age,Release_year,Total_ratings_per_user,Female,Male,Academic/educator,Artist,Clerical/admin,College/grad student,Customer service,Doctor/health care,Executive/managerial,Farmer,Homemaker,K-12 student,Lawyer,Other or not specified,Programmer,Retired,Sales/marketing,Scientist,Self-employed,Technician/engineer,Tradesman/craftsman,Unemployed,Writer,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Favourite_Action,Favourite_Adventure,Favourite_Animation,Favourite_Children's,Favourite_Comedy,Favourite_Crime,Favourite_Documentary,Favourite_Drama,Favourite_Fantasy,Favourite_Film-Noir,Favourite_Horror,Favourite_Musical,Favourite_Mystery,Favourite_Romance,Favourite_Sci-Fi,Favourite_Thriller,Favourite_War,Favourite_Western
0,5616,3590,2,2000,5,24,2,4,1974,130,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,4060,21,3,2000,8,5,15,2,1995,256,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,1125,3273,1,2000,11,22,16,1,2000,580,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,3410,585,3,2000,8,27,22,3,1995,731,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,3675,1374,3,2000,8,15,18,3,1982,849,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


# Random Forest

## Try 1



In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [28]:
# Define feature and target
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

In [29]:
# Split data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=42)

In [30]:
model = RandomForestClassifier()

In [31]:
# Define the parameter grid for hyperparameter optimization
param_dist = {
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [32]:
# Inner cross-validation loop for hyperparameter tuning (3 folds)
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Outer cross-validation loop for model evaluation (5 folds)
outer_cv = KFold(n_splits=5, shuffle=True, random_state= 42)

# Define classifier
clf = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                         cv=inner_cv, n_iter=50, random_state=42)


In [33]:
# Initialize lists to store metrics
accuracies = []
precisions = []
recalls = []
f1s = []

In [ ]:
# Perform nested cross-validation
for train_index, val_index in outer_cv.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    clf.fit(X_train_fold, y_train_fold)

    # Get the best model and predict on validation data
    best_model = clf.best_estimator_
    y_val_pred = best_model.predict(X_val_fold)

    # Calculate evaluation metrics
    accuracies.append(accuracy_score(y_val_fold, y_val_pred))
    precisions.append(precision_score(y_val_fold, y_val_pred, average='weighted'))
    recalls.append(recall_score(y_val_fold, y_val_pred, average='weighted'))
    f1s.append(f1_score(y_val_fold, y_val_pred, average='weighted'))

# Fit the final model on the entire training data and evaluate on the test set
best_clf = clf.fit(X_train, y_train)
y_test_pred = best_clf.predict(X_test)

# Print the results of the nested cross-validation
print(f'Nested CV accuracy (validation data): {np.mean(accuracies):.2f} ± {np.std(accuracies):.2f}')
print(f'Nested CV precision (validation data): {np.mean(precisions):.2f} ± {np.std(precisions):.2f}')
print(f'Nested CV recall (validation data): {np.mean(recalls):.2f} ± {np.std(recalls):.2f}')
print(f'Nested CV F1-score (validation data): {np.mean(f1s):.2f} ± {np.std(f1s):.2f}')

# Print test set results
print(f'Test set accuracy: {accuracy_score(y_test, y_test_pred):.2f}')
print(f'Test set precision: {precision_score(y_test, y_test_pred, average="weighted"):.2f}')
print(f'Test set recall: {recall_score(y_test, y_test_pred, average="weighted"):.2f}')
print(f'Test set F1-score: {f1_score(y_test, y_test_pred, average="weighted"):.2f}')

# Print the best parameters from hyperparameter tuning
print(f'Best parameters found: {clf.best_params_}')


# Gradient boosting

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Split the data into features (X) and target (y)
X = df_final.drop('Rating', axis=1)
y = df_final['Rating']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [ ]:
# Define the Gradient Boosting model
model = GradientBoostingClassifier()

In [ ]:
# Define the hyperparameter grid for Random Search
param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
# Inner cross-validation loop for hyperparameter tuning (3 folds)
inner_cv = KFold(n_splits=3, shuffle=True, random_state=1)

# Outer cross-validation loop for model evaluation (5 folds)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=1)

# Define classifier
clf = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                         cv=inner_cv, n_iter=50, random_state=42)

In [ ]:
# Initialize lists to store metrics
accuracies = []
precisions = []
recalls = []
f1s = []

In [ ]:
# Perform nested cross-validation
for train_index, val_index in outer_cv.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    clf.fit(X_train_fold, y_train_fold)

    # Get the best model and predict on validation data
    best_model = clf.best_estimator_
    y_val_pred = best_model.predict(X_val_fold)

    # Calculate evaluation metrics
    accuracies.append(accuracy_score(y_val_fold, y_val_pred))
    precisions.append(precision_score(y_val_fold, y_val_pred, average='weighted'))
    recalls.append(recall_score(y_val_fold, y_val_pred, average='weighted'))
    f1s.append(f1_score(y_val_fold, y_val_pred, average='weighted'))

# Fit the final model on the entire training data and evaluate on the test set
best_clf = clf.fit(X_train, y_train)
y_test_pred = best_clf.predict(X_test)

# Print the results of the nested cross-validation
print(f'Nested CV accuracy (validation data): {np.mean(accuracies):.2f} ± {np.std(accuracies):.2f}')
print(f'Nested CV precision (validation data): {np.mean(precisions):.2f} ± {np.std(precisions):.2f}')
print(f'Nested CV recall (validation data): {np.mean(recalls):.2f} ± {np.std(recalls):.2f}')
print(f'Nested CV F1-score (validation data): {np.mean(f1s):.2f} ± {np.std(f1s):.2f}')

# Print test set results
print(f'Test set accuracy: {accuracy_score(y_test, y_test_pred):.2f}')
print(f'Test set precision: {precision_score(y_test, y_test_pred, average="weighted"):.2f}')
print(f'Test set recall: {recall_score(y_test, y_test_pred, average="weighted"):.2f}')
print(f'Test set F1-score: {f1_score(y_test, y_test_pred, average="weighted"):.2f}')

# Print the best parameters from hyperparameter tuning
print(f'Best parameters found: {clf.best_params_}')
